# Notebook to update sample prices data

In [11]:
from pathlib import Path

import yfinance as yf # type: ignore


In [12]:
ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)

In [2]:
from functools import lru_cache

INTERVAL = dict(daily="1d", hourly="1h", minute="1m")
MAXPERIOD = dict(daily="max", hourly="2Y", minute="5d")


@lru_cache
def get_prices(symbol: str, *, freq="daily", max_bars=0):
    interval = INTERVAL[freq]
    period = MAXPERIOD[freq]
    prices = yf.Ticker(symbol).history(interval=interval, period=period, auto_adjust=True)
    prices = prices.filter(["Open", "High", "Low", "Close", "Volume"])
    prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)
    
    if freq == "daily":
        prices.index = prices.index.tz_localize(None)
    
    if max_bars > 0:
        prices = prices.tail(max_bars)
    return prices



In [3]:
SYMBOL = 'AAPL'
prices = get_prices(SYMBOL, freq="daily")
prices.index

In [5]:
for freq in INTERVAL.keys():
    prices = get_prices(SYMBOL, freq=freq, max_bars=5000)
    print(freq, len(prices), prices.index.dtype)


daily 5000 datetime64[ns]
hourly 3491 datetime64[ns, America/New_York]
minute 1949 datetime64[ns, America/New_York]


In [7]:
SAMPLES = PKGDIR.joinpath("samples")

for freq in INTERVAL.keys():
    fname = f"{freq}-prices.csv"
    prices = get_prices(SYMBOL, freq=freq)
    data = prices.to_csv(lineterminator="\n")
    outfile = SAMPLES.joinpath(fname)
    print(f"Updating {outfile.name} ...")
    outfile.write_text(data)

Updating daily-prices.csv ...
Updating hourly-prices.csv ...
Updating minute-prices.csv ...


In [8]:
from mintalib.samples import sample_prices

for freq in INTERVAL.keys():
    prices = sample_prices(freq=freq, max_bars=5000)
    print(freq, len(prices))


daily 5000
hourly 3491
minute 1949
